In [15]:
import sys
from pymystem3 import Mystem
mystem = Mystem(entire_input=False)

In [16]:
def tokenize_only(text):
    res = mystem.analyze(text)
    return list(map(lambda x: x.get('text').lower(), res))

def tokenize_and_stem(text):
    return mystem.lemmatize(text)

In [17]:
text = "Против губернатора Кировской области Никиты Белых возбуждено уголовное дело в связи с получением им взятки в особо крупном размере"

## Пример запуска токенизатора и стеминга

In [18]:
tokens = tokenize_only(text)
stems = tokenize_and_stem(text)
print("tokens: " + ", ".join(tokens))
print("stems: " + ", ".join(stems))

tokens: против, губернатора, кировской, области, никиты, белых, возбуждено, уголовное, дело, в, связи, с, получением, им, взятки, в, особо, крупном, размере
stems: против, губернатор, кировский, область, никита, белый, возбуждать, уголовный, дело, в, связь, с, получение, они, взятка, в, особо, крупный, размер


## Сырые данные о морфологии, которые выдает MyStem

In [19]:
print("full: ")
print(mystem.analyze(text))

full: 
[{'text': 'Против', 'analysis': [{'gr': 'PR=', 'lex': 'против'}]}, {'text': 'губернатора', 'analysis': [{'gr': 'S,муж,од=(вин,ед|род,ед)', 'lex': 'губернатор'}]}, {'text': 'Кировской', 'analysis': [{'gr': 'A=(пр,ед,полн,жен|дат,ед,полн,жен|род,ед,полн,жен|твор,ед,полн,жен)', 'lex': 'кировский'}]}, {'text': 'области', 'analysis': [{'gr': 'S,жен,неод=(пр,ед|вин,мн|дат,ед|род,ед|им,мн)', 'lex': 'область'}]}, {'text': 'Никиты', 'analysis': [{'gr': 'S,имя,муж,од=(род,ед|им,мн)', 'lex': 'никита'}]}, {'text': 'Белых', 'analysis': [{'gr': 'S,фам,муж,од=(пр,мн|вин,мн|род,мн)', 'lex': 'белый'}]}, {'text': 'возбуждено', 'analysis': [{'gr': 'V=прош,ед,прич,кр,сред,сов,страд,пе', 'lex': 'возбуждать'}]}, {'text': 'уголовное', 'analysis': [{'gr': 'A=(вин,ед,полн,сред|им,ед,полн,сред)', 'lex': 'уголовный'}]}, {'text': 'дело', 'analysis': [{'gr': 'S,сред,неод=(вин,ед|им,ед)', 'lex': 'дело'}]}, {'text': 'в', 'analysis': [{'gr': 'PR=', 'lex': 'в'}]}, {'text': 'связи', 'analysis': [{'gr': 'S,жен,не

## Попробуем поиграться c TfidfVectorizer

In [20]:
import numpy as np
import pandas as pd
import json
import stemming
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
with open('data/news.json', encoding="utf8") as f:
    news = json.load(f)

texts = list(map(lambda x: x['text'], news['news']))
print(texts)

['Инцидент произошёл с турбовинтовым самолётом Bombardier DHC-8 Q400, выполнявшим рейс номер 1899 из Аомори в Саппоро, передаёт РИА Новости. На его борту находились 40 человек, во время посадки никто не пострадал.', '«По словам девочки, 18 числа туристическая группа на двух лодках и плоту попала в шторм на озере Сямозеро, в результате чего группу раскидало по водоёму», — сообщил источник. По его данным, девочку выбросило на берег, где она пролежала без сознания до сегодняшнего дня. После того как девочка пришла в себя, она отправилась по берегу в сторону посёлка Кудама. «По дороге она обнаружила ещё одного мальчика, который был весь в крови и не мог передвигаться», - отмечает источник. 12-летняя москвичка запомнила место и отправилась за помощью в посёлок, где и встретила местного жителя, который позвонил спасателям.', '«Действительно, подтверждается, что эти (инструкторы. — RT) были, так скажем, юноши-инструкторы из педагогического колледжа петрозаводского, которые проходили там практ

In [22]:
tfidf_vectorizer = TfidfVectorizer(max_df=1, min_df=0.1, use_idf=True,
                                   tokenizer=stemming.tokenize_and_stem)

tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
idf = tfidf_vectorizer.idf_

In [23]:
#print (dict(zip(tfidf_vectorizer.get_feature_names(), idf)))
#print (tfidf_vectorizer.get_feature_names())
print (tfidf_matrix[0:3].todense())
print(tfidf_vectorizer.get_feature_names())

[[ 0.26726124  0.26726124  0.          0.          0.          0.26726124
   0.          0.          0.          0.          0.          0.          0.
   0.26726124  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.26726124
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.26726124  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.26726124  0.26726124  0.          0.          0.          0.          0.
   0. 

## K-Means

In [24]:
from sklearn.cluster import KMeans


In [47]:
num_clusters = 5

km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)

clusters = km.cluster_centers_
labels = km.labels_

In [48]:
print(labels)
# print(clusters)

[1 0 4 1 2 1 3 1 1 1]
